In [7]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from backprop import Value

In [23]:
a = Value(2.0)
b = 3.0

In [ ]:
# understanding derivatives
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
# let's plot the function to determine shape (parabola)
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
# now calculate derivatives (3*2*x - 4)
h = 0.001
x = 3.0
(f(x + h) - f(x))/h

In [ ]:
# bit more complex:
def f(a, b, c):
    return a * b + c

a = 2.0
b = -3.0
c = 10.0
print(f(a, b, c))

In [ ]:
h = 0.001
d1 = f(a, b, c)
c += h
d2 = f(a, b, c)
print('d1', d1)
print('d2', d2)

In [ ]:
print('slope', (d2 - d1)/h)

In [ ]:
math.exp(1)

In [ ]:
from graphviz import Digraph

def trace(root):
    # build a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir})

    nodes, edges = trace(root)
    
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph create a rectangular ('record') node for it
        dot.node(name=uid, label="{ %s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of operation create op node for it
            dot.node(name=uid + n._op, label=n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 node to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [ ]:
h = 0.001
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b
e.label='e'
d = e + c
d.label = 'd'
f = Value(-2.0, label='f')
L = d*f
L.label='L'

draw_dot(L)

### Manual backpropagation

In [ ]:
# https://www.youtube.com/watch?v=VMj-3S1tku0&ab_channel=AndrejKarpathy

# if we change L by h we expect the entire function to change by h so the derivative should be 1
# proof:
# dL/dL = 1
# function changes linearly

# for * node we can determine gradients this way:
# L = d * f
# dL/dd =? f
# change d (x) by tiny amount h:
# (f(x+h)-f(x))/h = (fx + fh - fx)/h = fh/h = f

# this will mean:
# f.grad = d
# d.grad = f


L.grad = 1
d.grad = -2.0
f.grad = 4.0

def lol():
    h = 0.001
    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b
    e.label='e'
    d = e + c
    d.label = 'd'
    f = Value(-2.0, label='f')
    L = d*f
    L.label='L'
    L1 = L.data

    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b
    e.label='e'
    d = e + c
    d.label = 'd'
    d.data += h # for gardient check... numerical verification that we calculated f.grade correctly
    f = Value(-2.0, label='f')
    L = d*f
    L.label='L'
    L2 = L.data

    print((L2-L1) / h)

lol()

draw_dot(L)


In [ ]:
'''
next we need to understand dL by dc, i.e. dL/dc
focusing on d only we know that L get impacted by c through d so what is the impact of c on d?
dd / dc = ?
we know that d = c + e so dd/dc = 1.0
proof (input c+e for x):
(f(x+h) - f(x))/h = (c + h + e) - (c + e) / h = h/h = 1

dd / de = 1.0 - this is just local node derivative and we need to continue

WANT:
dL by dc i.e. dL / dc

KNOW:
dL / dd

applying chain rule:
dL/de = dL/dd * dd/de = -2.0 * 1.0 = -2.0
'''
c.grad = -2.0
e.grad = -2.0 # explain why :-)

draw_dot(L)

In [ ]:
'''
let's do the final part here finding dL / db and dL / da
we know that e = a * b
first we will do de / da which we expect to be b
and symetrically de / db = a

these are just local so we need to continue applying chain rule knowing that e = a * b

WANT:
dL / da

KNOW:
dL / dd; dL / de; dd/de but we just need the node we are backpropagating from i.e. dL/de and 
we are not traversing across entire chain

WRONG: dL/dd * dd/de * de/da
RIGHT: dL/de * de/da

applying chain rule:
dL / da = dL/de * de/da = -2.0 * b = -3.0 == 6.0
dl / db = dL/de * de/db = -2.0 * a = 2.0 == -4
'''
a.grad = 6.0
b.grad = -4.0
draw_dot(L)

In [ ]:
'''
Let's increase the value of some leaf nodes by 0.01 to see impact on L
'''
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

e = a * b
d = e + c
L = d * f

print(L)

### Manual backpropagation example with neuron

In [ ]:
# inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#bias
b = Value(6.8813735870195432, label='b')
# according to neuron model (wikipedia)
# x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1w1 + x2w2'
n = x1w1x2w2 + b; n.label = 'n'

In [ ]:
# take this through the activation function tanh
# tanh we need to implement in Value class (check code above)
o = n.tanh(); o.label = 'o'

In [ ]:
draw_dot(o)

In [ ]:
# starting the backpropagation
# STEP 1: what is the derivative of o with respect to o? it's 1
o.grad = 1
draw_dot(o)

In [ ]:
# STEP 2: if o = tanh(n) what is do/dn?
# do/dn = 1 - (tanh(n**2)) = 1 - o**2
n.grad = 1 - o.data**2
draw_dot(o)

In [ ]:
# STEP 3: What is do / db
# WE KNOW that + operation just routes next gradient so 0.5 will go both into b and x1w1+x2w2
b.grad = 0.5
x1w1x2w2.grad = 0.5
draw_dot(o)

In [ ]:
# STEP 4 - same as STEP3
x1w1.grad = 0.5
x2w2.grad = 0.5
draw_dot(o)

In [ ]:
# STEP 5
# what id do/dx1 ? 
# we know that x1w1 = x1 * w1 so dx1w1/dx1 = w1 and dx1w1/w1 = x1

# apply chain rule : do/dx1 = do/dx1w1 * dx1w1/dx1 (which is w1) = 0.5 * -3.0
# and also : do/dw1 = do/dx1w1 * dx1w1/dw1 (which is x1) = 0.5 * 2
x1.grad = 0.5 * -3.0
w1.grad = 1.0


# similarly x2.grad = do/dx2?
# do/dx2 = do/dw2x2 * dw2x2/dx2 (which is w2) so it means that x2.grad = x2w2.grad * w2.data
x2.grad = 0.5 * 1.0
w2.grad = 0.5 * 0
draw_dot(o)

To implement backpropagation now we go back to Value object and create self._backward = lambda: None
After implementation of _backward functions we can re-run the backpropagation to check if it works

In [ ]:
# inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#bias
b = Value(6.8813735870195432, label='b')
# according to neuron model (wikipedia)
# x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1w1 + x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

In [ ]:
# initialize o.grad
o.grad = 1.0
o._backward()
draw_dot(o)

In [ ]:
n._backward()
draw_dot(o)

In [ ]:
x1w1x2w2._backward()
draw_dot(o)

In [ ]:
x1w1._backward()
x2w2._backward()
draw_dot(o)

In [ ]:
# building topological sort of nodes so we don't have to call backward function numerous times
topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(o)
topo

In [ ]:
# inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#bias
b = Value(6.8813735870195432, label='b')
# according to neuron model (wikipedia)
# x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1w1 + x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

o.backward()


In [ ]:
draw_dot(o)

In [ ]:
a = 2
b = Value (2.0)
a * b

In [ ]:
# inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#bias
b = Value(6.8813735870195432, label='b')
# according to neuron model (wikipedia)
# x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1w1 + x2w2'
n = x1w1x2w2 + b; n.label = 'n'

# break tanh
e = (2*n).exp()
o = (e - 1)/(e + 1)
o.label = 'o'

o.backward()
draw_dot(o)

In [ ]:
import random

# nin = number of inputs
class Neuron:
    def __init__(self, nin) -> None:
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)] # weights
        self.b = Value(random.uniform(-1, 1)) # bias

    def __call__(self, x):
        # w * x + b -> forward pass
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.tanh()
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)] # a layer is just a list of neurons

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        # return [p for neuron in self.neurons for p in neuron.parameters()]
        params = []
        for neuron in self.neurons:
            ps = neuron.parameters()
            params.extend(ps)
        return params
        
# multi layer perceptron
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts # nouts is a list of layers containing neurons
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        
        return x
    
    def parameters(self):
        # return [p for layer in self.layers for p in layer.parameters()]
        params = []
        for layer in self.layers:
            ps = layer.parameters()
            params.extend(ps)
        return params

x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

In [ ]:
# draw_dot(n(x))

In [ ]:
# let's create a simple dataset with 4 possible inputs into the nn
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
] 

ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
# compute MDE:
ypred = [n(x) for x in xs]
loss = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
total_loss = Value(0.0)
for l in loss:
    total_loss += l
total_loss

In [ ]:
total_loss.backward()

In [ ]:
n.layers[0].neurons[0].w[0].grad

In [ ]:
# draw_dot(total_loss)

In [ ]:
# gather all parameters of NN:
n.parameters()

In [ ]:
len(n.parameters())

In [ ]:
print(n.layers[0].neurons[0].w[0].grad)
print(n.layers[0].neurons[0].w[0].data)

In [ ]:
# now we want to start training and use gradient descen to train our NN
# we want to change data vector in the direction of the gradient

# update
for p in n.parameters():
    p.data += -0.01 * p.grad

In [ ]:
print(n.layers[0].neurons[0].w[0].grad)
print(n.layers[0].neurons[0].w[0].data)

In [ ]:
# compute MDE - forward pass
ypred = [n(x) for x in xs]
loss = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
total_loss = Value(0.0)
for l in loss:
    total_loss += l
total_loss

In [ ]:
# now we just need to iterate the process

# backward pass
for p in n.parameters():
    p.grad = 0.0
total_loss.backward()

In [ ]:
ypred

In [ ]:
n.parameters() # setting of parameters that makes our predictions very precise